<img src="data/images/lecture-notebook-header.png" />

# Preparation: Data Collection Using Web Scraping

Web scraping is the automated process of extracting data from websites. It involves writing code to access web pages, retrieve the HTML content, and extract specific information from the page, such as text, images, links, or structured data.

The purpose of web scraping in data mining is to gather data from various websites in a structured and systematic manner. It allows organizations and individuals to collect large amounts of data from the web for analysis, research, or other purposes. Here are some key purposes of web scraping in data mining:

* **Data Collection:** Web scraping enables the extraction of data from multiple sources on the internet, including websites, online directories, social media platforms, e-commerce sites, and more. It automates the data collection process, saving time and effort compared to manual data entry.

* **Market Research:** Web scraping can be used to gather market data, such as product prices, customer reviews, competitor information, and trends. This information helps businesses analyze the market landscape, identify pricing strategies, monitor competitors, and make informed decisions.

* **Sentiment Analysis:** By scraping data from social media platforms, forums, or review sites, web scraping can be used to gather user-generated content and perform sentiment analysis. This analysis helps in understanding public opinion, identifying customer preferences, and monitoring brand reputation.

* **Lead Generation:** Web scraping can be employed to extract contact information, such as email addresses or phone numbers, from websites. This data can be used for lead generation purposes, customer acquisition, or marketing campaigns.

* **Data Integration:** Web scraping allows the aggregation of data from different websites into a single database or system. This integration enables the combination of disparate data sources and enhances data analysis and reporting capabilities.

* **Price Comparison and Monitoring:** E-commerce websites often change their prices dynamically. Web scraping can be utilized to gather product information and prices from multiple online stores, enabling price comparison and monitoring. This information helps consumers find the best deals and assists businesses in adjusting their pricing strategies.

It is important to note that while web scraping offers valuable opportunities for data mining, it is crucial to respect website terms of service and legal requirements. Always ensure you comply with applicable laws and respect the website's policies when scraping data.

Web Scraping typically involves two main steps: (a) fetching/downloading the web page and (b) extracting the relevant information. A web page is commonly a HTML document containing paragraphs, images, tables, and all kinds of content, and the relevant content can be found somewhere in this structure.

To extract the right content from a web page using Python, [`BeautifulSoup`](https://pypi.org/project/beautifulsoup4/) is an excellent package; downloading the web pages in the first place is even easier with Python. In this notebook, we go through a very basic example. As use case we scrape the names of all the shopping malls in Singapore from Wikipedia.

## Setting up the Notebook

### Import Required Packages

In [ ]:
import pandas as pd
import requests
import json

from bs4 import BeautifulSoup

---

## Getting All Shopping Mall in Singapore

On Wikipedia, you can find a list of all shopping malls in Singapore:

* [https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore](https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore)

In the following, we will try to extract all the names and store it into a text file.

### Fetching the Web Page


In [ ]:
URL = 'https://en.wikipedia.org/wiki/List_of_shopping_malls_in_Singapore'

The [`requests`](https://pypi.org/project/requests/) library makes it very easy to fetch the HTML content of web page given its URL:

In [ ]:
html = requests.get(URL).text

Wikipedia is very "easy-going" when it comes to scraping. Many other sites prefer that their content is only requested from genuine browsers. To do this, they check the header information sent with the request -- some websites are even more restrictive. However, this is very easy to fix by explicitly setting this header information.

In [ ]:
# This is just an example mimicking the header sent by a browser
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

html = requests.get(URL, headers=headers).text

Now our request "looks" to Wikipedia as if it came from a browser. But as you saw above, Wikipedia does not really care about it. For many other sites, this is a common issue though.

Anyway, now we have the whole HTML content of this Wikipedia page -- calling `text` on the request response returns the HTML content as a string. So we can have a look:

In [ ]:
print(html[:600])

The names of the shopping malls are somewhere in this string. This brings up to the second main step of Web Scraping, extraction of the relevant information.

### Information Extraction

First, we convert the HTML string into a `BeautifulSoup` object. This will allow us to navigate and search the HTML document.

In [ ]:
soup = BeautifulSoup(html,'html.parser')

You can print the `soup` variable, which basically just show the complete HTML code again.

In [ ]:
#print(soup)

The powerful part of `BeautifulSoup` is that we now can search for specific parts in the HTML document. However, this requires what we are looking for. For this, we have to take a closer look at the web page. We look at the list of shopping malls, we can see that each mall is listed with a bullet point.

To really see how the names of shopping malls are presented, we have to look more closely at the HTML code. The easiest way is to use the feature to inspect an element of choice. Below shows the context menu of Firefox when clicking right on the first name of a shopping mall in the Wikipedia page. The important menu entry is "Inspect Element" (in Chrome, it should be only "Inspect").

![title](data/images/screenshot-wikipedia-01.png)

Selecting "Inspect Element" opens a view showing the HTML content, focused on the selected element (i.e., the element that contains the name of the first listed shopping mall).

![title](data/images/screenshot-wikipedia-02.png)

This confirms that each shopping mall is listed using `<li>...</li>` tags. Of course, the whole page may contain main pieces of information that are not the name of shopping malls but also rendered in `<li>` tags. While this is not the case in this example, let's pretend it is.

We therefore have to find to further filter the extracted information to reduce the risk of irrelevant content. Here we notice that all lists of shopping mall names are child elements of a `<div class="div-col">`. So we can first select only those `<div>` elements before diving deeper into the `<li>` tags.

In [ ]:
divs = soup.find_all('div', {'class': 'div-col'})

print('Number of DIV elements of class "div-col": {}'.format(len(divs)))

If we look at the Wikipedia page, we can see that there a 6 blocks containing the list of shopping mall names: Central, East, North, North East, North West, South, and West. This means that we already excluded all the HTML content outside those block we are interested in.

Now let's look at the first `<div>`:

In [ ]:
first_div = divs[0]

`first_div` is again a `BeautifulSoup` object we can use to further find the information we want. We already know that we need all `<li>` tags, which is equally easy to do.

In [ ]:
li_tags = first_div.find_all('li')

print('Number of LI tags in the first DIV: {}'.format(len(li_tags)))

As the first `<div>` represents the Central region, this means that there are 68 shopping malls in that region. Again looking at the Wikipedia page, that looks about right.

The only thing that is left is to get the names of the shopping malls. We can see that some shopping mall names contain links pointing to their own Wikipedia page. However, we do not have to bother since we are only interested in the text node if the `<li>` HTML element. Easy enough to do with `BeautifulSoup`.

In [ ]:
for tag in li_tags:
    print(tag.text)

Of course, we can put this all into nested loops to fetch all the names and store them into a file. Although it is a bit overkill for this simple example, a good practice is to store the data into a `pandas` DateFrame and then export the DataFrame to a file.

Let's first get the names of all the shopping malls. We can basically reuse the code from above. To make it a little bit more interesting, let's also get the Wikipedia link for shopping malls that have their own Wikipedia article.

In [ ]:
names, links = [], []

for div in divs:
    li_tags = div.find_all('li')
    for tag in li_tags:
        # First the name
        name = tag.text
        names.append(name)
        # Then the link
        link = tag.find('a') # li is a again a BeautifulSoup object which we can use to search for <a> tags
        if link is None:
            links.append(None) # not all shopping malls are linked to their own page
        else:
            links.append(link['href']) # link is a BeautifulSoup object and we want to attribute 'href'
        
print('We now have list of {} names of shopping malls.'.format(len(names)))

Now we can take this list of names and create a `pandas` dataframe from it. Since it is just a single column, it looks a bit boring, of course.

In [ ]:
df = pd.DataFrame(list(zip(names, links)), columns=['name', 'link'])

df.head(10)

Note that the links are just relative links, but that's not a problem as one can easily add `https://en.wikipedia.org` to each relative link later on if needed.

Lastly, we can export the DataFrame to a CSV file using the in-built methods provided by `pandas`. Note that the separator is set to `sep='\t'`. This avoids any issues in case a shopping mall name contains a comma. Although it is not the case here, such are common issues when dealing with real-world data.

In [ ]:
df.to_csv('data/datasets/singapore/sg-shopping-malls.csv', sep='\t')

You can open the file with any text editor to see its content.

### Further Data Collection

Having all the names of shopping malls and the optional Wikipedia link does not mean the end of the data collection. For example, we can now use the names as a query string to fetch the geolocation of each mall using the Nominatim API (see the other notebook) -- which in fact was done for complete the auxiliary data for the Kaggle competition. Or, we can scrape further information of shopping malls from the Wikipedia pages of which we have the links now.

---

## Summary

Web Scraping, particular when using the right tools or packages, is a reasonably straightforward task. However, there are a number of challenges or even obstacles in the way:

* **Manual effort:** As we have seen above, extracting the relevant information requires inspecting the HTML source code to find the right elements needed. In our example, this was pretty easy, but can get very tricky in practice and might require a lot of additional filtering or rules/heuristics to get rid of irrelevant bits in the HTML content.

* **Maintenance:** Sometimes it is necessary to scrape the same web page periodically (e.g., to fetch the latest updates). Any change of the layout and the HTML structure might break the scraper. For example, let's say the shopping malls on the Wikipedia page will no longer be listed using `<li>` tags but in one or more tables. In this case, the code above will no longer work.

* **Dynamic pages:** Most modern websites load the content dynamically, e.g., via Javascript/Ajax request -- that is, the initial request fetches Javascript code that gets executed by the browser to fetch more content. Packages such as `requests` cannot execute Javascript. As such, the received HTML content will only contain the information about Javascript code (potentially just the URL of a Javascript file to be loaded). While there are solutions involving Javascript engines to scrape such dynamic websites, the required efforts quickly explode.

* **Legal issues:** There's a long-standing debate if it is legal to collect the content from websites. I'm not a lawyer but I hope that Wikipedia does not cause any problems. We also do not use the data for commercial purposes. For further information, I recommend the following [article](https://singaporelegaladvice.com/law-articles/legal-scrape-crawl-websites-data-singapore/).

In short, using an API is always the preferred method to collect data. Of course, this assumes that a platform provides such an API.